## Translation Alignment using NLTK

## Alignment
A storage class for representing alignment between two sequences, $s_1$, $s_2$. In general, an alignment is a set of tuples of the form $(i, j)$ representing an alignment between the $i$-th element of $s_1$ and the $j$-th element of $s_2$.

In [ ]:
from nltk.translate import Alignment

a = Alignment([(0, 0), (0, 1), (1, 2), (2, 2)])
a.invert() # Reversing the directionality

In [ ]:
# Read a giza-formatted string and return an Alignment object.
Alignment.fromstring('0-0 2-1 9-2 21-3 10-4 7-5') 

## AlignedSent
`AlignedSent` encapsulates two sentences along with an ``Alignment`` between them. Typically used in machine translation to represent a sentence and its translation.

In [ ]:
from nltk.translate.api import AlignedSent

algnsent = AlignedSent(['klein', 'ist', 'das', 'Haus'],
                       ['the', 'house', 'is', 'small'], 
                       Alignment.fromstring('0-3 1-2 2-0 3-1'))

In [ ]:
algnsent.words # Words in the target language sentence

In [ ]:
algnsent.mots # Words in the source language sentence

In [ ]:
algnsent.alignment

In [ ]:
def printAlignment(als):
    print(" ".join(als.words))
    print(" ".join(als.mots))
    print("\n".join([als.words[a[0]]+" == "+als.mots[a[1]] for a in als.alignment 
                     if isinstance(a[0], int) and isinstance(a[1], int) ]))

In [ ]:
printAlignment(algnsent)

### NLTK Aligned Corpus
NLTK makes available as the comtrans corpus a subset of Europarl's sentence-aligned English/French, German/English and German/French data, approximately 33,000 sentence pairs in each case.

In [ ]:
from nltk.corpus import comtrans
fileids = comtrans.fileids()
fileids

In [ ]:
for id in fileids:
    print(id +" "+ str(len(comtrans.sents(id))))

In [ ]:
words = comtrans.words('alignment-en-fr.txt')
words[:20]

In [ ]:
als = comtrans.aligned_sents('alignment-en-fr.txt')[0]
als.words

In [ ]:
printAlignment(als)

In [ ]:
printAlignment(als.invert())

## IBM Models

The IBM models are a series of generative models that learn lexical translation probabilities, 

`P(target language word|source language word)` ,

given a sentence-aligned parallel corpus.

The models increase in sophistication from model 1 to 5. Typically, the output of lower models is used to seed the higher models. 

All models use the <b>Expectation-Maximization (EM)</b> algorithm to learn various probability tables.

### Preprocessing
we will use the Bible parallel corpus.

In [ ]:
from nltk.tokenize import RegexpTokenizer

# read the data from
f = open("data/txt/English.txt", "r")
lines = f.readlines()
f.close()
# split
for line in lines:
    temp = line.split("\t") # b.GEN.1.1	In the beginning God created the heaven and the earth.
    # indexing & tokenization
    english[temp[0]] = tokenizer.tokenize(temp[1].strip().lower())
english

In [ ]:
f = open("data/txt/German.txt", "r")
lines = f.readlines()
f.close()
for line in lines:
    temp = line.split("\t") # b.GEN.1.1	Am Anfang schuf Gott Himmel und Erde.
    # indexing & tokenization
    german[temp[0]] = tokenizer.tokenize(temp[1].strip().lower())
german

In [ ]:
# merge the two files in one parallel corpus i.e. a list of AlignedSent
parallelSentences = [AlignedSent(s, english[id]) for id, s in german.items() if id in english]
parallelSentences[:10]

### IBM Model 1
In IBM Model 1, word order is ignored for simplicity. As long as the word alignments are equivalent, it doesn’t matter where the word occurs in the source or target sentence. Thus, the following three alignments are equally likely.
- Source: je mange du jambon
- Target: i eat some ham
- Alignment: (0,0) (1,1) (2,2) (3,3)


- Source: je mange du jambon
- Target: some ham eat i
- Alignment: (0,2) (1,3) (2,1) (3,1)


- Source: du jambon je mange
- Target: eat i some ham
- Alignment: (0,3) (1,2) (2,0) (3,1)

<div class="alert alert-info">
    The EM algorithm used in Model 1 is:

- <b>E step</b> - In the training data, count how many times a source language word is translated into a target language word, weighted by the prior probability of the translation.

- <b>M step</b> - Estimate the new probability of translation based on the counts from the Expectation step.
</div>

In [ ]:
from nltk.translate import IBMModel1
em_ibm1 = IBMModel1(corpus[:300], 20)

In [ ]:
d = em_ibm1.translation_table['geist']
sorted(d.items(), key=lambda x: x[1],reverse=True)[:5]

In [ ]:
d = em_ibm1.translation_table['wasser']
sorted(d.items(), key=lambda x: x[1],reverse=True)[:5]

In [ ]:
d = em_ibm1.translation_table['gott']
sorted(d.items(), key=lambda x: x[1],reverse=True)[:5]

In [ ]:
printAlignment(corpus[0])

### IBM Model 2
Lexical translation model that considers word order. IBM Model 2 improves on Model 1 by accounting for word order. An alignment probability is introduced, a(i | j,l,m), which predicts a source word position, given its aligned target word’s position.

Notations:
- $i$: Position in the source sentence 
    (Valid values are 0 (for NULL), 1, 2, ..., length of source sentence)
- $j$: Position in the target sentence 
    (Valid values are 1, 2, ..., length of target sentence)
- $l$: Number of words in the source sentence, excluding NULL
- $m$: Number of words in the target sentence

<div class="alert alert-info">
    The EM algorithm used in Model 2 is:

- <b>E step</b> - In the training data, collect counts, weighted by prior probabilities.

(a) count how many times a source language word is translated into a target language word

(b) count how many times a particular position in the source sentence is aligned to a particular position in the target sentence

- <b>M step</b> - Estimate new probabilities based on the counts from the E step
</div>

In [ ]:
from nltk.translate import IBMModel2
em_ibm2 = IBMModel2(corpus[:300], 20)

In [ ]:
printAlignment(corpus[2])

In [ ]:
printAlignment(corpus[0])

In [ ]:
# nltk.align.gale_church

In [ ]:
d = em_ibm2.translation_table['licht']
sorted(d.items(), key=lambda x: x[1],reverse=True)[:5]

In [ ]:
d = em_ibm1.translation_table['licht']
sorted(d.items(), key=lambda x: x[1],reverse=True)[:5]